## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x114d9f2b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x114d9ce80>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
model.invoke([HumanMessage(content="Hi, My name is Vatsal. I am a AI Engineer. What is your name?")])

AIMessage(content="Hello Vatsal, it's nice to meet you! I am Gemma, an open-weights AI assistant. I don't have a last name. 😊\n\nWhat can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 29, 'total_tokens': 75, 'completion_time': 0.083636364, 'prompt_time': 0.001401549, 'queue_time': 0.188514311, 'total_time': 0.085037913}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c38180e6-5161-4e5d-af98-97d51cc461cb-0', usage_metadata={'input_tokens': 29, 'output_tokens': 46, 'total_tokens': 75})

In [9]:
messages = [HumanMessage(content="Hi, My name is Vatsal. I am a AI Engineer. What is your name?"),
AIMessage(content="Hello Vatsal, I am an AI model created by Groq. How can I assist you today?"),
HumanMessage(content="What is your favorite programming language?")]

model.invoke(messages)

AIMessage(content="As a large language model, I don't have personal preferences like a favorite programming language. I can process and understand many different languages, including Python, Java, C++, and more.\n\nWhich language do you enjoy working with the most? I'd love to learn more about it!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 68, 'total_tokens': 130, 'completion_time': 0.112727273, 'prompt_time': 0.002277108, 'queue_time': 0.188888188, 'total_time': 0.115004381}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--71090893-4520-4956-80e2-674753d880fe-0', usage_metadata={'input_tokens': 68, 'output_tokens': 62, 'total_tokens': 130})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [16]:
store={}

def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_history = RunnableWithMessageHistory(model, get_chat_history)

In [17]:
config={"configurable":{"session_id":"chat1"}}

In [21]:
response=with_history.invoke(
    [HumanMessage(content="Hi, My name is Vatsal. I am a AI Engineer. What is your name?")],
    config=config
)

In [22]:
response

AIMessage(content="Hi Vatsal, it's nice to meet you!\n\nMy name is Gemma. I'm an AI assistant, here to help with any questions or tasks you might have.  \n\nWhat kind of AI work are you involved in?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 103, 'total_tokens': 156, 'completion_time': 0.096363636, 'prompt_time': 0.002852148, 'queue_time': 0.18865328, 'total_time': 0.099215784}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--19e2a00e-7ece-4a53-80c4-e347b2bab12a-0', usage_metadata={'input_tokens': 103, 'output_tokens': 53, 'total_tokens': 156})

In [23]:
with_history.invoke([HumanMessage(content="What is my name?")], 
config=config)

AIMessage(content='Your name is Vatsal.  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 169, 'total_tokens': 191, 'completion_time': 0.04, 'prompt_time': 0.003931258, 'queue_time': 0.18789434700000002, 'total_time': 0.043931258}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a021654a-acfa-406b-ae9d-ae2dd47a828f-0', usage_metadata={'input_tokens': 169, 'output_tokens': 22, 'total_tokens': 191})

In [24]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it!\n"

In [26]:
response=with_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Nice to meet you, John! 👋 \n\nHow can I help you today? 😊\n'

In [27]:
response=with_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, remember?  😊  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [32]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Vatsal")]})

AIMessage(content="Hello Vatsal! It's nice to meet you. \n\nI'm happy to answer any questions you have to the best of my ability.  What can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 32, 'total_tokens': 79, 'completion_time': 0.085454545, 'prompt_time': 0.001485859, 'queue_time': 0.18880281899999998, 'total_time': 0.086940404}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ce1ac286-2748-4a5a-a287-27c17d879230-0', usage_metadata={'input_tokens': 32, 'output_tokens': 47, 'total_tokens': 79})

In [33]:
store={}

def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_history = RunnableWithMessageHistory(model, get_chat_history)

In [34]:
config={"configurable":{"session_id":"chat3"}}

response=with_history.invoke({"messages":[HumanMessage(content="Hi My name is Vatsal")]},
    config=config
)

response.content

"Hi Vatsal, it's nice to meet you! 👋\n\nWhat can I do for you today? 😊  \n\n"

In [36]:
response=with_history.invoke({"messages":[HumanMessage(content="What is my name?")]},
    config=config
)
response.content

'Your name is Vatsal.  \n\nI remember! 😉  Would you like to talk about something else?\n'

In [37]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [39]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Vatsal")],"language":"Hindi"})
response.content

'नमस्ते वत्सल! 👋 \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। \n\nआप मुझसे क्या पूछना चाहेंगे?  😊\n\n'

In [41]:
with_history=RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="messages"
)

In [43]:
config={"configurable":{"session_id":"chat4"}}

repsonse=with_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Vatsal")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते वत्सल! 👋  मुझे ख़ुशी है कि आप मुझसे बात कर रहे हैं। \n\nआप मुझसे क्या जानना चाहेंगे? 😊 \n'

In [44]:
response = with_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [45]:
response.content

'आपका नाम वत्सल है! 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [47]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [48]:
trimmer.invoke(messages)

/Users/vatsal/Machine Learning/Gen AI/LangChain/lang/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [51]:
from operator import itemgetter
def get_last_n_messages(messages, n):
    return messages[-n:]

result= get_last_n_messages(messages, 5)
result

[AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [53]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [56]:
from langchain_core.runnables import RunnablePassthrough

chain= (RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
 | prompt 
 | model)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What was my last 3 message?")],
    "language":"English"
    }
)
response.content

'Your last three messages were:\n\n1. thanks\n2. no problem!\n3. having fun?'

In [57]:
## Lets wrap this in the MEssage History
config={"configurable":{"session_id":"chat5"}}
with_history=RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="messages"
)


In [59]:
response = with_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats was the math problem?")],
        "language": "English",
    },
    config=config,
)

response.content

'The math problem was 2 + 2. \n\nIs there anything else I can help you with? 😊\n'